In [ ]:
import pandas as pd
import pygmt
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2


In [ ]:
#df_SAM_merge = pd.read_csv('/Users/simon/GIT/bx/andes//datafiles/geochem_merge_20230104.csv')
df_SAM_merge = pd.read_csv('/Users/simon/GIT/bx/andes//datafiles/geochem_merge_20231117.csv')
df_SAM_merge = gpd.GeoDataFrame(df_SAM_merge, geometry=gpd.points_from_xy(df_SAM_merge.Longitude, df_SAM_merge.Latitude), crs=4326)

df_SAM_merge = df_SAM_merge[~np.isnan(df_SAM_merge.age)]


In [ ]:
gdf = gpd.read_file('/Users/simon/GIT/Tectonic/global_tectonics-main/plates&provinces/global_gprv_wage.shp')

gdf = gdf[gdf['continent']=='South America']

cordillera = gdf[gdf['lastorogen'].str.contains('Cordilleran', na=False)]
terra_australis = gdf[gdf['lastorogen'].str.contains('Australis', na=False)]

cordillera = cordillera.cx[-100:-50, -60:20]
terra_australis = terra_australis.cx[-100:-50, -60:20]

df_SAM_merge = df_SAM_merge.clip(pd.concat([cordillera, terra_australis]))

df_SAM_merge

In [ ]:
source_studies = df_SAM_merge.source_study.unique()
#source_studies.sort

colors = sns.color_palette(None, len(source_studies))

source_studies = sorted(list(source_studies))

source_studies


In [ ]:
region = [-90,-20,-58,14]


topo_grid = pygmt.datasets.load_earth_relief(resolution='02m', region=region)
dgrid = pygmt.grdgradient(grid=topo_grid, radiance=[290, 10])

style='t0.12c'
transparency = 50

fig = pygmt.Figure()
fig.basemap(projection='R16c', region=region, frame='afg')

pygmt.makecpt(cmap="gray", series=[-5000, 3000]) #, reverse=True)
fig.grdimage(grid=topo_grid, cmap=True)

pygmt.makecpt(cmap="gray", series=[-1.5, 0.3, 0.01])
fig.grdimage(
    grid=dgrid,
    cmap=True,
    transparency=80,
)

fig.coast(water='lightblue', transparency=50)

#fig.plot(x=subset.geometry.x, y=subset.geometry.y, style=style, pen='1p,red', transparency=transparency, label='Samples')
for study_name,color in zip(source_studies, colors):
    subset = df_SAM_merge[df_SAM_merge['source_study']==study_name]
    fig.plot(x=subset.geometry.x, y=subset.geometry.y, style=style, 
             pen='1p,{:d}/{:d}/{:d}'.format(int(color[0]*255),int(color[1]*255),int(color[2]*255)), 
             transparency=transparency, label=study_name)

fig.legend(position="JTR+jTR+o0.2c", box='+gwhite+p1p,grey', transparency=30)
fig.savefig('../images/sam_basemap.png', transparent=True)
fig.show(width=800)


In [ ]:
region = [-85,-55,-57,16]
projection = 'W2.7c'

fig = pygmt.Figure()
with fig.subplot(nrows=2, ncols=7, figsize=("21c", "15.7c"), frame="lrtb"):
    for index,start_age in enumerate(np.arange(0,350,25)):  # row number starting from 0     
        with fig.set_panel(panel=index):  # sets the current panel
            
            fig.coast(land="lightgrey", region=region, projection=projection)

            subset2=df_SAM_merge[(df_SAM_merge.age>=start_age) & (df_SAM_merge.age<start_age+25.)]

            fig.plot(x=subset2.Longitude, y=subset2.Latitude, style='c0.15c', color='black',
                     region=region, projection=projection, transparency=50)
            
            fig.basemap(frame=['a','wesn'], region=region, projection=projection)
            
            fig.text(x=-70, y=14, text='{:0.0f}-{:0.0f} Ma'.format(start_age,start_age+25.), 
                     font='12p', no_clip=True, region=region, projection=projection)
            
fig.savefig('../images/sam_basemap_data_coverage_timeslices.png')

fig.show(width=1000)


In [ ]:
fig,ax = plt.subplots()
ax.hist(df_SAM_merge.age, bins=np.arange(0,351,10))
ax.set_yscale('log')
ax.grid()
ax.grid(which='both', axis='y', linestyle='--')
ax.set_xlim(0,350)
ax.set_xlabel('Sample Age [Ma]')
ax.set_ylabel('Number of Samples')
plt.savefig('../images/sam_data_coverage_time_histogram.png', dpi=600)
plt.show()


In [ ]:
pygmt.show_versions()

In [ ]:
#print(len(colors))
colors